### ✅ Face Detection
- Face Tracking 진행 예정!

### ✅ 1. 카메라 인스턴스 생성

In [1]:
from jetbot import Camera # 카메라 관련 라이브러리
from jetbot import bgr8_to_jpeg # bgr type -> jpeg 변환

camera = Camera.instance(width = 720, height = 720)

In [ ]:
# 오류?
# 1) 왼쪽 정지버튼 -> 전체 커널 셧다운
# 2) 커널 restart
# 3) 재부팅
# 4) 카메라 접촉 불량 ...

### ✅ 2. 카메라 프리뷰 생성

In [2]:
import ipywidgets.widgets as widgets #파이썬에서 위젯을 활용할 수 있는 라이브러리
from IPython.display import display #실제 이미지를 display할 수 있는 라이브러리

detect_face = widgets.Image(width = 300, height = 300, format = 'jpeg') # 인식될 공간

display(detect_face)

Image(value=b'', format='jpeg', height='300', width='300')

In [ ]:
# 카메라 테스트

while 1:
    detect_face.value = bgr8_to_jpeg(camera.value) # 카메라 값을 변환시킨다

### ✅ 학습된 모델을 가져오자!! 

- https://github.com/opencv/opencv/tree/master/data/haarcascades

In [3]:
# curl 명령어 통해서 링크로 학습된 데이터 가져오기!
# 터미널창 sudo apt-get install curl
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  908k  100  908k    0     0   187k      0  0:00:04  0:00:04 --:--:--  289k


In [4]:
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

face_x = face_y = face_w = face_h = 0 # 얼굴을 표시할 좌표를 0으로 초기화

In [ ]:
while 1:
    frame = camera.value
    
    cv2.resize(frame, (300, 300)) # 학습된 데이터의 이미지가 300*300 픽셀
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백으로 변경
    
    faces = face_cascade.detectMultiScale(gray) # 흑백 이미지 내에서 얼굴 검출
    
    if len(faces) > 0 : # 만약 검출된게 있다면
        (face_x, face_y, face_w, face_h) = faces[0] # 사진에 좌표 생성
        
        # 직사각형 그리기!
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 0, 255), 4) # 직사각형 생성 # (0, 0, 255) -> 빨강
    
    detect_face.value = bgr8_to_jpeg(frame)

### ✅ 눈 검출!
- ROT(Region of Interest)

In [5]:
# 눈 관련 학습 모델 가져오기!
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_eye.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  333k  100  333k    0     0   184k      0  0:00:01  0:00:01 --:--:-- 14.2M


In [6]:
import cv2

# 눈 관련 cascade 모델 생성!

eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# 눈 좌표 초기화

eye_x = eye_y = eye_w = eye_h = 0

In [ ]:
while 1:
    frame = camera.value
    
    cv2.resize(frame, (300, 300)) # 학습된 데이터의 이미지가 300*300 픽셀
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백으로 변경
    
    faces = face_cascade.detectMultiScale(gray) # 흑백 이미지 내에서 얼굴 검출
    
    if len(faces) > 0 : # 만약 검출된게 있다면
        (face_x, face_y, face_w, face_h) = faces[0] # 사진에 좌표 생성
        
        # 직사각형 그리기!
        
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 0, 255), 4)
        # 직사각형 생성 # (0, 0, 255) -> 빨강
        
        # 눈이 검출된다 -> 먼저 얼굴이 검출되어야 함
        
        roi_region = gray[face_y : face_y + face_h, face_x : face_x + face_w]
                     # -> gray 전체 프레임에서 얼굴 영역만 슬라이싱
            
        roi_color = frame[face_y : face_y + face_h, face_x : face_x + face_w]
                    # -> 눈을 표시할 수 있는 도형을 컬러 프레임에 찍기 위함! # 원본 -> frame
        
        eyes = eye_cascade.detectMultiScale(roi_region) # 눈 검출
        
        # 눈이 검출되었다면! 
        
        for eye_x, eye_y, eye_w, eye_h in eyes : # 눈이라고 인식된 모든 공간
            cv2.rectangle(roi_color, (eye_x, eye_y), (eye_x + eye_w, eye_y + eye_h), (255, 0, 0), 2)
            # (255, 0, 0) -> 파랑
    
    detect_face.value = bgr8_to_jpeg(frame)

In [1]:
from servoserial import ServoSerial

servo = ServoSerial()

serial Open!


In [ ]:
# chmod -> change mode
# 리눅스의 권한을 주는 방식
# shdo chmod 777 /dev/ttyTHS1
# ____ _____ _____ read, write, execute
# user group other
# 111 101 001 -> sudo chmod 751

In [2]:
# 내 로봇 머리의 중앙값 찾기
# 1번 인덱스 좌우(600 ~ 3600), 2번 인덱스 상하(1300 ~ 4095)

servo.Servo_serial_double_control(1, 2400, 2, 2100)

134
b'\xff\xff\xfe\x0e\x83*\x04\x01\t`\x00\n\x02\x084\x00\n\x86'


In [9]:
# 내 로봇 머리의 중앙 값

center_x = 2400
center_y = 2100

### ✅ PID(비례 - 적분 - 미분)
- 카메라가 얼굴 탐지 영역의 중심점을 찾아가는데 
- 어느정도로 움직여야할지 그 정도를 계산하는 라이브러리

In [ ]:
# P (propotional : 비례)
# - 현재 물체와 원하는 위치 사이의 오차에 비례해서 움직임을 조절
# -> 오차가 크면 많이 움직이고 적으면 조금 움직임 -> 오차를 빠르게 줄여주는 역할

# I (integral : 적분)
# - 오랜 시간 동안 물체가 원하는 위치에 도달하지 못했을 때, 이전 오차들을 더한 후 누적해서 물체를 움직임
# -> 오차를 완전히 없애주고 정확한 위치로 가게 함

# D (derivative : 미분)
# - 물체가 급격히 위치를 바꿀 때 이 변화를 미리 감지하고 조절
# -> 로봇이 너무 빨리 이동하지 않게 함

# -> 세가지 요소를 결합해서 로봇 머리가 사람 얼굴을 찾아 부드럽고 정확하게 따라가도록 함

In [10]:
import PID

xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

In [ ]:
while 1:
    frame = camera.value
    
    cv2.resize(frame, (300, 300)) # 학습된 데이터의 이미지가 300*300 픽셀
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백으로 변경
    
    faces = face_cascade.detectMultiScale(gray) # 흑백 이미지 내에서 얼굴 검출
    
    if len(faces) > 0 : # 만약 검출된게 있다면
        (face_x, face_y, face_w, face_h) = faces[0] # 사진에 좌표 생성
        
        # 직사각형 그리기!
        
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 0, 255), 4)
        # 직사각형 생성 # (0, 0, 255) -> 빨강
        
        # X축
        
        xservo_pid.SystemOutput = face_x + face_w / 2 # 얼굴의 x축 중심점
        xservo_pid.SetStepSignal(150) # 단계 신호 설정 -> 얼굴까지 얼마나 빠르게 움직여야 되는가! -> 150이면 빠른 정도
        xservo_pid.SetInertiaTime(0.01, 0.006) # 관성시간 -> 변화에 대해서 얼마나 빠르게 대응!
        target_x = int(center_x + xservo_pid.SystemOutput)
        
        # Y축
        yservo_pid.SystemOutput = face_y + face_h / 2 # 얼굴의 x축 중심점
        yservo_pid.SetStepSignal(150) # 단계 신호 설정 -> 얼굴까지 얼마나 빠르게 움직여야 되는가! -> 150이면 빠른 정도
        yservo_pid.SetInertiaTime(0.01, 0.006) # 관성시간 -> 변화에 대해서 얼마나 빠르게 대응!
        target_y = int(center_y + yservo_pid.SystemOutput)
        
        servo.Servo_serial_double_control(1, target_x, 2, target_y)
        
        # 눈이 검출된다 -> 먼저 얼굴이 검출되어야 함
        
        roi_region = gray[face_y : face_y + face_h, face_x : face_x + face_w]
                     # -> gray 전체 프레임에서 얼굴 영역만 슬라이싱
            
        roi_color = frame[face_y : face_y + face_h, face_x : face_x + face_w]
                    # -> 눈을 표시할 수 있는 도형을 컬러 프레임에 찍기 위함! # 원본 -> frame
        
        eyes = eye_cascade.detectMultiScale(roi_region) # 눈 검출
        
        # 눈이 검출되었다면! 
        
        for eye_x, eye_y, eye_w, eye_h in eyes : # 눈이라고 인식된 모든 공간
            cv2.rectangle(roi_color, (eye_x, eye_y), (eye_x + eye_w, eye_y + eye_h), (255, 0, 0), 2)
            # (255, 0, 0) -> 파랑
    
    detect_face.value = bgr8_to_jpeg(frame)

168
b"\xff\xff\xfe\x0e\x83*\x04\x01\x08L\x00\n\x02\x08'\x00\n\xa8"
7
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07:\x00\n\x02\x06\xdd\x00\n\x07'
247
b'\xff\xff\xfe\x0e\x83*\x04\x01\x06\x9b\x00\n\x02\x06\x8d\x00\n\xf7'
112
b'\xff\xff\xfe\x0e\x83*\x04\x01\x06\xb6\x00\n\x02\x06\xf9\x00\np'
224
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\x0c\x00\n\x02\x071\x00\n\xe0'
173
b'\xff\xff\xfe\x0e\x83*\x04\x01\x06\x94\x00\n\x02\x06\xde\x00\n\xad'
35
b'\xff\xff\xfe\x0e\x83*\x04\x01\x06r\x00\n\x02\x06\x8a\x00\n#'
47
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\x00\x00\n\x02\x06\xef\x00\n/'
172
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\x12\x00\n\x02\x07_\x00\n\xac'
76
b'\xff\xff\xfe\x0e\x83*\x04\x01\x05\xc1\x00\n\x02\x06\x13\x00\nL'
153
b'\xff\xff\xfe\x0e\x83*\x04\x01\x05i\x00\n\x02\x06\x1e\x00\n\x99'
44
b'\xff\xff\xfe\x0e\x83*\x04\x01\x04\xfe\x00\n\x02\x05\xf8\x00\n,'
